In [1]:
# a. Data preparation
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Lambda
import tensorflow.keras.backend as K
import random

# Example corpus (small for simplicity, use a larger one for better results)
corpus = [
    "the quick brown fox jumped over the lazy dog",
    "the dog barked at the fox",
    "the fox is quick and the dog is lazy",
    "the quick dog jumped over the lazy fox"
]

# Tokenize the corpus
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
word2idx = tokenizer.word_index
idx2word = {v: k for k, v in word2idx.items()}
vocab_size = len(word2idx) + 1  # Add 1 for padding (if needed)
print(f'Vocabulary Size: {vocab_size}')

# Convert text to sequences
sequences = tokenizer.texts_to_sequences(corpus)

# b. Generate training data
# Context window size: 2 words before and after the target word
window_size = 2

def generate_context_target_pairs(sequences, window_size):
    context_target_pairs = []
    for sequence in sequences:
        for i, word in enumerate(sequence):
            # Define the context window range
            start = max(0, i - window_size)
            end = min(len(sequence), i + window_size + 1)
            context = [sequence[j] for j in range(start, end) if j != i]
            target = word
            if len(context) == window_size * 2:
                context_target_pairs.append((context, target))
    return context_target_pairs

# Generate the training data (context and target pairs)
pairs = generate_context_target_pairs(sequences, window_size)

# Split context and target words
contexts, targets = zip(*pairs)
contexts = np.array(contexts)
targets = np.array(targets)

# One-hot encode the target words
targets = to_categorical(targets, num_classes=vocab_size)

# c. Train the model

# Define CBOW Model
embedding_dim = 100

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=window_size * 2))
model.add(Lambda(lambda x: K.mean(x, axis=1)))  # Average embeddings for context words
model.add(Dense(vocab_size, activation='softmax'))  # Output softmax over vocabulary

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy')

# Train the model
model.fit(contexts, targets, epochs=100, verbose=1)

# d. Output (Predict target word from context)
# Predict the target word for a given context

def predict_target(context_words):
    # Convert words to their respective indices
    context_indices = [word2idx[word] for word in context_words]
    context_indices = np.array(context_indices).reshape(1, window_size * 2)
    
    # Predict the target word's index
    predicted_idx = np.argmax(model.predict(context_indices), axis=1)[0]
    
    # Convert index back to word
    return idx2word[predicted_idx]

# Example: Predict the target word from the context ['quick', 'brown', 'jumped', 'over']
context_words = ['quick', 'brown', 'jumped', 'over']
predicted_word = predict_target(context_words)
print(f"Context: {context_words} -> Predicted Target Word: '{predicted_word}'")



Vocabulary Size: 13


Epoch 1/100

1/1 [==============================] - 1s 994ms/step - loss: 2.5725
Epoch 2/100
1/1 [==============================] - 0s 6ms/step - loss: 2.5672
Epoch 3/100
1/1 [==============================] - 0s 8ms/step - loss: 2.5619
Epoch 4/100
1/1 [==============================] - 0s 9ms/step - loss: 2.5567
Epoch 5/100
1/1 [==============================] - 0s 6ms/step - loss: 2.5515
Epoch 6/100
1/1 [==============================] - 0s 5ms/step - loss: 2.5462
Epoch 7/100
1/1 [==============================] - 0s 6ms/step - loss: 2.5410
Epoch 8/100
1/1 [==============================] - 0s 5ms/step - loss: 2.5358
Epoch 9/100
1/1 [==============================] - 0s 6ms/step - loss: 2.5305
Epoch 10/100
1/1 [==============================] - 0s 6ms/step - loss: 2.5253
Epoch 11/100
1/1 [==============================] - 0s 7ms/step - loss: 2.5201
Epoch 12/100
1/1 [==============================] - 0s 7ms/step - loss: 2.5148
Epoch 13/100
1/1 [=================

1/1 [==============================] - 0s 8ms/step - loss: 1.8627
Epoch 95/100
1/1 [==============================] - 0s 5ms/step - loss: 1.8519
Epoch 96/100
1/1 [==============================] - 0s 3ms/step - loss: 1.8412
Epoch 97/100
1/1 [==============================] - 0s 9ms/step - loss: 1.8304
Epoch 98/100
1/1 [==============================] - 0s 6ms/step - loss: 1.8195
Epoch 99/100
1/1 [==============================] - 0s 6ms/step - loss: 1.8087
Epoch 100/100
1/1 [==============================] - 0s 128ms/step
Context: ['quick', 'brown', 'jumped', 'over'] -> Predicted Target Word: 'the'
